In [ ]:
!pip install trl==0.13.0
!pip install datasets==3.2.0
!pip install -U bitsandbytes==0.45.0
!pip install git+https://github.com/huggingface/peft.git -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.3 MB/s eta 0:00:00
  In

In [ ]:
from datasets import Dataset
import transformers
from peft import prepare_model_for_kbit_training
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pickle
import random
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# 이전에 생성한 질문과 답변 데이터셋인 'qa_data.pkl' 파일을 불러옵니다.
data_list = pickle.load(open('qa_data.pkl', 'rb'))

In [ ]:
# 사용할 사전 학습된 인스트럭션 모델 ID를 지정합니다.
model_id = "aifeifei798/DarkIdol-Llama-3.1-8B-Instruct-1.2-Uncensored"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/923 [00:00<?, ?B/s]

...

model-00007-of-00009.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00009.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# 학습 스크립트 형식을 정의합니다. 질문에 답변할 때 참고할 문서 두 개와 질문 및 답변 형식을 지정합니다.
script_format = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

너는 제공된 문서를 참조하여 답변을 하는 사람이야.<|eot_id|><|start_header_id|>user<|end_header_id|>
- 문서 1
{doc_1}
- 문서 2
{doc_2}

질문 : {q}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
답변 : {a} <|eot_id|>
"""

In [ ]:
# 네거티브 샘플링을 적용하여 RAG 학습 데이터를 생성하는 함수
def create_rag_data_with_negative_sampling(data_list):
    rag_data = []

    for data in data_list:
        q = data['question']  # 질문
        a = data['answer']    # 정답 답변
        correct_doc = data['chunk']  # 정답 문서

        # 정답 문서를 제외한 다른 문서에서 네거티브 샘플을 하나 선택
        negative_samples = random.sample([d['chunk'] for d in data_list if d['chunk'] != correct_doc], 1)

        # 정답 문서와 네거티브 문서를 무작위로 배치
        documents = [correct_doc] + negative_samples
        random.shuffle(documents)

        # 정답 문서의 위치를 1 또는 2로 설정
        correct_doc_position = documents.index(correct_doc) + 1

        # script_format을 사용해 학습용 스크립트를 생성
        script = script_format.format(
            doc_1=documents[0],
            doc_2=documents[1],
            q=q,
            a=a
        )

        # 생성된 데이터를 리스트에 추가
        rag_data.append({
            'script': script,
            'correct_doc_position': correct_doc_position
        })

    return rag_data

In [ ]:
# RAG 학습 데이터를 생성
# 생성된 데이터는 정답과 네거티브 샘플이 포함됨
rag_data_with_negatives = create_rag_data_with_negative_sampling(data_list)

In [ ]:
data = Dataset.from_list([{"text" : i['script']} for i in rag_data_with_negatives], split="train")

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 375
})

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
test_script = rag_data_with_negatives[5]['script'].split('답변 : ')[0]
test_script_answer = rag_data_with_negatives[5]['script'].split('ssistant<|end_header_id|>\n')[1]

gened = model.generate(
      **tokenizer(
          test_script,
          return_tensors='pt',
          return_token_type_ids=False
      ).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=[128001,128009],
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(test_script)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

너는 제공된 문서를 참조하여 답변을 하는 사람이야.<|eot_id|><|start_header_id|>user<|end_header_id|>
- 문서 1
8. 제37조제3항제5호의 규정을 위반하여 전자화폐등에 의한 거래를 대행한 자
9. 제37조제4항의 규정을 위반하여 가맹점의 이름으로 전자화폐등에 의한 거래를 한 자
10. 허위 그 밖의 부정한 방법으로 전자금융거래정보를 열람하거나 제공받은 자
⑥다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 1천만원 이하의 벌금에 처한다.<개정 2020. 5.
19.>
1. 삭제<2008. 12. 31.>
2. 삭제<2013. 5. 22.>
3. 제37조제1항의 규정을 위반하여 전자화폐등에 의한 거래를 이유로 재화 또는 용역의 제공을 거절하거나 이용자
를 불리하게 대우한 자 
4. 제37조제2항의 규정을 위반하여 이용자에게 가맹점수수료를 부담하게 한 자
5. 제37조제3항제4호의 규정을 위반하여 가맹점의 이름을 타인에게 빌려준 자
6. 제45조제1항의 규정에 따른 인가를 받지 아니하고 동항 각 호의 어느 하나에 해당하는 행위를 한 자
⑦ 제1항제1호ㆍ제2호 및 제3호와 제2항제1호ㆍ제2호 및 제4호의 미수범은 처벌한다.<개정 2014. 10. 15., 2020. 5.
19.>
⑧제1항부터 제7항까지의 징역형과 벌금형은 병과할 수 있다.<개정 2020. 5. 19.>
 
제49조(벌칙) ① 다음 각 호의 어느 하나에 해당하는 자는 10년 이하의 징역 또는 1억원 이하의 벌금에 처한다. <신설
2014. 10. 15.>
1. 제21조의4제1호를 위반하여 전자금융기반시설에 접근하거나 저장된 데이터를 조작ㆍ파괴ㆍ은닉 또는 유출한 자
2. 제21조의4제2호를 위반하여 데이터를 파괴하거나 컴퓨터 바이러스, 논리폭탄 또는 메일폭탄 등의 프로그램을 투
입한 자 
3. 제21조의4제3호를 위반하여 일시에 대량의 신호

In [ ]:
# 학습된 모델을 사용하여 질문에 대한 답변을 생성하고, 생성된 답변과 정답을 비교
print(' -- 모델 답변')
print(tokenizer.decode(gened[0]).split('<|start_header_id|>assistant<|end_header_id|>\n')[1])
print(' -- 정답 답변')
print(test_script_answer)

 -- 모델 답변
가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관<|eot_id|>
 -- 정답 답변
답변 : 전자금융업자는 「전자금융거래법」 제28조에 따라 허가를 받거나 등록을 한 자로서, 금융회사를 제외한 기관입니다. 즉, 금융회사가 아닌 기관이 전자금융거래를 할 수 있도록 허가를 받은 경우를 말합니다. <|eot_id|>



In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_len = data.num_rows

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=200,
        num_train_epochs = 3,
        # max_steps= int(data_len/batch_size) , # 1 epoch 만 학습
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="qlora_output",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.815400
20,1.693400
30,1.705300
40,1.761100
50,1.628100
60,1.582900
70,1.494500
80,1.431700
90,1.432400
100,1.270900


KeyboardInterrupt: 

In [ ]:
gened = model.generate(
      **tokenizer(
          test_script,
          return_tensors='pt',
          return_token_type_ids=False
      ).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=128001,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
print(' -- 모델 답변')
print(tokenizer.decode(gened[0]).split('<|start_header_id|>assistant<|end_header_id|>\n')[1])
print(' -- 정답 답변')
print(test_script_answer)